## Skorch RHC network classification example

In [1]:
import subprocess

# Installation on Google Colab
try:
    import google.colab
    subprocess.run(['python', '-m', 'pip', 'install', 'skorch' , 'torch'])
except ImportError:
    pass

In [2]:
import numpy as np
from sklearn.datasets import make_classification
import torch
from torch import nn
from skorch import NeuralNetClassifier
from skorch import NeuralNet
from pyperch.neural.rhc_nn import RHCModule  
from pyperch.utils.decorators import add_to
from skorch.dataset import unpack_data
import copy

In [3]:
X, y = make_classification(1000, 12, n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)
print(X.shape, y.shape)
input_dim=12
output_dim=2
hidden_units=20
hidden_layers=1
lr=.05

(1000, 12) (1000,)


In [4]:
rhc_module=RHCModule(input_dim=input_dim, output_dim=output_dim, hidden_units=hidden_units, hidden_layers=hidden_layers, lr=lr)

net = NeuralNetClassifier(
    rhc_module,
    max_epochs=500,
    #lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

## train_step_single override - add RHC training step and disable backprop 

In [5]:
RHCModule.register_rhc_training_step()

In [6]:
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6749       0.5350        0.6924  0.0184
      2        0.6768       0.5350        0.6908  0.0233
      3        0.6750       0.5350        0.6889  0.0241
      4        0.6745       0.5350        0.6890  0.0264
      5        0.6745       0.5350        0.6890  0.0203
      6        0.6728       0.5300        0.6857  0.0288
      7        0.6696       0.5450        0.6850  0.0260
      8        0.6686       0.5350        0.6849  0.0233
      9        0.6684       0.5450        0.6846  0.0264
     10        0.6684       0.5500        0.6859  0.0280
     11        0.6684       0.5500        0.6857  0.0199
     12        0.6679       0.5500        0.6846  0.0209
     13        0.6677       0.5500        0.6845  0.0270
     14        0.6672       0.5750        0.6821  0.0197
     15        0.6664       0.5700        0.6816  0.0249
     16        0.6662       0.5

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=RHCModule(
    (layers): ModuleList(
      (0): Linear(in_features=12, out_features=20, bias=True)
      (1): Linear(in_features=20, out_features=20, bias=True)
      (2): Linear(in_features=20, out_features=2, bias=True)
    )
    (activation): ReLU()
    (dropout): Dropout(p=0, inplace=False)
    (softmax): Softmax(dim=-1)
  ),
)

## Using sklearn pipeline with RO

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipe = Pipeline([
    ('scale', StandardScaler()),
    ('net', net),
])

pipe.fit(X, y)
y_proba = pipe.predict_proba(X)

Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6863       0.5150        0.6941  0.0296
      2        0.6862       0.5150        0.6938  0.0276
      3        0.6860       0.5200        0.6935  0.0238
      4        0.6859       0.5150        0.6938  0.0214
      5        0.6858       0.5100        0.6939  0.0217
      6        0.6859       0.4900        0.6947  0.0260
      7        0.6860       0.4950        0.6945  0.0262
      8        0.6855       0.5050        0.6943  0.0235
      9        0.6854       0.5000        0.6941  0.0246
     10        0.6853       0.5100        0.6928  0.0247
     11        0.6835       0.5350        0.6905  0.0258
     12        0.6832       0.5350        0.6901  0.0282
     13        0.6820       0.5350        0.6896  0.0353
     14        0.6819       0.5350        0.6897  0.0495
     15   

## Using sklearn grid search with RO

In [8]:
from sklearn.model_selection import GridSearchCV

# deactivate skorch-internal train-valid split and verbose logging
net.set_params(train_split=False, verbose=0, )

default_params = {
    'module__input_dim': [12],
    'module__output_dim': [2],
}

grid_search_params = {
    'lr': [0.01, 0.02],
    'max_epochs': [10, 20],
    'module__hidden_units': [10, 20],
    **default_params,
}

gs = GridSearchCV(net, grid_search_params, refit=False, cv=3, scoring='accuracy', verbose=2)

gs.fit(X, y)
print("best score: {:.3f}, best params: {}".format(gs.best_score_, gs.best_params_))

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END lr=0.01, max_epochs=10, module__hidden_units=10, module__input_dim=12, module__output_dim=2; total time=   0.2s
[CV] END lr=0.01, max_epochs=10, module__hidden_units=10, module__input_dim=12, module__output_dim=2; total time=   0.2s
[CV] END lr=0.01, max_epochs=10, module__hidden_units=10, module__input_dim=12, module__output_dim=2; total time=   0.2s
[CV] END lr=0.01, max_epochs=10, module__hidden_units=20, module__input_dim=12, module__output_dim=2; total time=   0.2s
[CV] END lr=0.01, max_epochs=10, module__hidden_units=20, module__input_dim=12, module__output_dim=2; total time=   0.2s
[CV] END lr=0.01, max_epochs=10, module__hidden_units=20, module__input_dim=12, module__output_dim=2; total time=   0.2s
[CV] END lr=0.01, max_epochs=20, module__hidden_units=10, module__input_dim=12, module__output_dim=2; total time=   0.3s
[CV] END lr=0.01, max_epochs=20, module__hidden_units=10, module__input_dim=12, module__outpu